<a href="https://colab.research.google.com/github/hanumantjain/agentic-ai/blob/main/Building_a_Quasi_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install litellm --quiet

import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [8]:
from litellm import completion
from typing import List, Dict

def generate_response(messages: List[Dict]) -> str:
  response= completion(
      model='openai/gpt-4o-mini',
      messages= messages,
      max_tokens=512
  )
  return response.choices[0].message.content
what_function_to_create = input('What function to create: ')
messages=[
    {'role':'system', 'content': 'You are an expert software engineer who uses python that generates functions and gives separate code from commentry'},
    {'role':'user', 'content':what_function_to_create},
]

response = generate_response(messages)
print(response)

messages.append({'role':'assistant', "content":response})
messages.append({'role':'user', 'content':'Add comprehensive documentation including function description, parameter description, return value, example and edge cases'})

response = generate_response(messages)
print(response)

messages.append({'role':'assistant', 'content':response})
messages.append({'role':'user', 'content':'Add the test cases using Python unittest framework which includes basic functionality, edge cases, error cases and various input scenarios'})

response = generate_response(messages)
print(response)

What function to create: check prime number
Here's a Python function to check if a number is prime:

```python
def is_prime(n):
    """Check if a number is prime."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True
```

### Commentary
1. **Function Definition**: The function `is_prime` takes a single argument `n`, which is the number to be checked.
2. **Basic Checks**: 
   - If `n` is less than or equal to 1, it returns `False` since such numbers are not prime.
   - If `n` is 2 or 3, it returns `True` as both are prime.
3. **Divisibility Checks**: 
   - It checks if `n` is divisible by 2 or 3 and returns `False` if so.
4. **Loop for Larger Numbers**: 
   - It uses a loop starting from 5 and checks divisibility with `i` and `i + 2` (i.e., checks 5, 7, 11, 13, ...), continuing until `